# DATA EXPLORATION NOTEBOOK

### **I. Import thư viện và tải dữ liệu**

**Import thư viện**

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest, shapiro, pearsonr, spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import warnings
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

**Cấu hình**

In [29]:
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

INPUT_FOLDER = "data"
INPUT_FILE = "football_players_dataset.csv"

**Lấy dữ liệu từ file**

In [30]:
file_path = os.path.join(INPUT_FOLDER, INPUT_FILE)
df = pd.read_csv(file_path)

print(f"\nKích thước Dataset:")
print(f"   - Số lượng bản ghi (dòng): {len(df):,}")
print(f"   - Số lượng thuộc tính (cột): {len(df.columns)}")
print(f"   - Tổng số ô dữ liệu: {df.shape[0] * df.shape[1]:,}")
print(f"   - Dung lượng bộ nhớ: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n10 dòng đầu tiên")
df.head(10)


Kích thước Dataset:
   - Số lượng bản ghi (dòng): 16,453
   - Số lượng thuộc tính (cột): 46
   - Tổng số ô dữ liệu: 756,838
   - Dung lượng bộ nhớ: 10.27 MB

10 dòng đầu tiên


,age,nationality,position,position_category,current_club,league,appearances,minutes_played,is_GK,is_DF,...,passes_received_per90,interceptions_per90,blocks_per90,ball_recoveries_per90,aerials_won_per90,yellow_cards_per90,fouls_committed_per90,calculated_mpg,calculated_sum,market_value
0,23.0,New Zealand,GK,GK,Auckland FC,A-League Men,26,2340,1,0,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,90.000000,0.00,0.75
1,27.0,Uruguay,FW,FW,Auckland FC,A-League Men,26,2268,0,0,...,28.280,0.48,0.81,3.55,2.59,0.00,2.10,87.230769,0.33,0.75
2,31.0,New Zealand,DF,DF,Auckland FC,A-League Men,26,2198,0,1,...,34.055,1.09,1.14,4.11,1.43,0.20,1.03,84.538462,0.00,0.60
3,26.0,New Zealand,DF,DF,Auckland FC,A-League Men,25,2142,0,1,...,34.055,1.09,1.14,4.11,1.43,0.20,1.03,85.680000,0.00,0.45
4,35.0,Japan,DF-MF,DF,Auckland FC,A-League Men,23,2024,0,1,...,38.170,2.00,1.33,3.00,2.83,0.17,1.00,88.000000,0.00,0.50
5,27.0,New Zealand,FW-MF,MF,Auckland FC,A-League Men,26,1782,0,0,...,31.760,0.67,1.00,4.54,0.74,0.19,1.29,68.538462,0.00,0.45
6,26.0,Belgium,MF,MF,Auckland FC,A-League Men,23,1734,0,0,...,32.260,2.10,1.61,7.10,0.81,0.32,2.26,75.391304,0.15,0.60
7,27.0,Australia,MF,MF,Auckland FC,A-League Men,25,1498,0,0,...,31.760,0.67,1.00,4.54,0.74,0.19,1.29,59.920000,0.00,0.60
8,35.0,New Zealand,DF,DF,Auckland FC,A-League Men,24,1213,0,1,...,23.270,0.78,1.38,4.33,2.28,0.11,0.60,50.541667,0.21,0.10
9,23.0,New Zealand,FW,FW,Auckland FC,A-League Men,13,939,0,0,...,22.700,0.20,0.64,2.48,2.12,0.17,1.50,72.230769,0.00,0.35


### **II. Khám phá dữ liệu**

#### **1. Ý nghĩa từng cột dữ liệu**

| STT | Cột                              | Ý nghĩa                                                      |
|-----|----------------------------------|--------------------------------------------------------------|
| 1   | age                              | Tuổi hiện tại của cầu thủ                                     |
| 2   | nationality                      | Quốc tịch của cầu thủ                                         |
| 3   | position                          | Vị trí cụ thể trên sân (ST, CM, CB...)                       |
| 4   | position_category                 | Nhóm vị trí chung (GK/DF/MF/FW)                              |
| 5   | current_club                      | Câu lạc bộ hiện tại                                          |
| 6   | league                            | Giải đấu mà cầu thủ thi đấu                                   |
| 7   | appearances                       | Số trận đã thi đấu                                           |
| 8   | minutes_played                    | Tổng số phút đã thi đấu                                       |
| 9   | is_GK                             | Chỉ số nhị phân thủ môn                                       |
| 10  | is_DF                             | Chỉ số nhị phân hậu vệ                                        |
| 11  | is_MF                             | Chỉ số nhị phân tiền vệ                                       |
| 12  | is_FW                             | Chỉ số nhị phân tiền đạo                                      |
| 13  | goals                             | Tổng số bàn thắng                                            |
| 14  | assists                           | Tổng số kiến tạo                                             |
| 15  | npg_per90                          | Bàn thắng không tính penalty mỗi 90 phút                      |
| 16  | xag_per90                          | Expected assisted goals mỗi 90 phút                            |
| 17  | xg_per90                           | Expected goals mỗi 90 phút                                     |
| 18  | shots_per90                        | Số cú sút mỗi 90 phút                                         |
| 19  | shots_on_target_per90              | Số cú sút trúng đích mỗi 90 phút                               |
| 20  | shots_on_target_pct                | Tỷ lệ cú sút trúng đích                                       |
| 21  | avg_shot_distance                  | Khoảng cách trung bình của cú sút                               |
| 22  | sca_per90                          | Hành động tạo cơ hội sút mỗi 90 phút                            |
| 23  | gca_per90                          | Hành động tạo cơ hội ghi bàn mỗi 90 phút                        |
| 24  | key_passes_per90                   | Số đường chuyền quan trọng mỗi 90 phút                          |
| 25  | passes_completed_per90             | Số đường chuyền thành công mỗi 90 phút                          |
| 26  | pass_completion_pct                | Tỷ lệ chuyền bóng thành công                                    |
| 27  | passes_into_final_third_per90      | Số đường chuyền vào 1/3 sân cuối mỗi 90 phút                    |
| 28  | passes_into_penalty_area_per90     | Số đường chuyền vào khu vực 16m50 mỗi 90 phút                    |
| 29  | progressive_passes_per90           | Số đường chuyền tiến triển mỗi 90 phút                           |
| 30  | progressive_passes_rec_per90       | Số đường chuyền tiến triển nhận được mỗi 90 phút                |
| 31  | progressive_carries_per90          | Số lần dẫn bóng tiến triển mỗi 90 phút                           |
| 32  | take_ons_per90                     | Số lần đi bóng qua người mỗi 90 phút                              |
| 33  | carries_per90                      | Số lần dẫn bóng mỗi 90 phút                                       |
| 34  | carries_into_final_third_per90     | Số lần dẫn bóng vào 1/3 sân cuối mỗi 90 phút                      |
| 35  | touches_att_third_per90            | Số lần chạm bóng trong 1/3 sân cuối mỗi 90 phút                   |
| 36  | touches_att_pen_per90              | Số lần chạm bóng trong khu vực 16m50 mỗi 90 phút                   |
| 37  | passes_received_per90              | Số lần nhận bóng mỗi 90 phút                                      |
| 38  | interceptions_per90                | Số lần cắt bóng mỗi 90 phút                                       |
| 39  | blocks_per90                       | Số lần chặn bóng mỗi 90 phút                                       |
| 40  | ball_recoveries_per90              | Số lần giành lại bóng mỗi 90 phút                                  |
| 41  | aerials_won_per90                  | Số lần thắng tranh chấp trên không mỗi 90 phút                     |
| 42  | yellow_cards_per90                 | Số thẻ vàng nhận được mỗi 90 phút                                   |
| 43  | fouls_committed_per90              | Số lỗi phạm phải mỗi 90 phút                                        |
| 44  | calculated_mpg                     | Số phút trung bình mỗi trận                                         |
| 45  | calculated_sum                     | Trường tính tổng (kết hợp các chỉ số)                              |
| 46  | market_value                       | Giá trị thị trường của cầu thủ (triệu €)                             |


#### **2. Kiểu dữ liệu từng cột**

In [31]:
dtype_summary = pd.DataFrame({
    'Cột': df.columns,
    'Kiểu_dữ_liệu': df.dtypes.astype(str).values,
    'Không_null': df.notna().sum().values,
    'Số_null': df.isna().sum().values,
    'Null_%': (df.isna().sum().values / len(df) * 100).round(2),
    'Giá_trị_duy_nhất': [df[col].nunique() for col in df.columns],
    'Mẫu': [str(df[col].dropna().iloc[0])[:30] if df[col].notna().any() else 'N/A' 
            for col in df.columns]
})

print("\nTỔNG HỢP CHI TIẾT KIỂU DỮ LIỆU:")
print("-" * 90)
print(dtype_summary.to_string(index=False))

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()


TỔNG HỢP CHI TIẾT KIỂU DỮ LIỆU:
------------------------------------------------------------------------------------------
                           Cột Kiểu_dữ_liệu  Không_null  Số_null  Null_%  Giá_trị_duy_nhất          Mẫu
                           age      float64       16451        2    0.01                30         23.0
                   nationality       object       16215      238    1.45               163  New Zealand
                      position       object       16450        3    0.02                11           GK
             position_category       object       16453        0    0.00                 5           GK
                  current_club       object       16453        0    0.00               811  Auckland FC
                        league       object       16453        0    0.00                57 A-League Men
                   appearances        int64       16453        0    0.00                50           26
                minutes_played        int64 

#### **3. Phân bố kiểu dữ liệu**

In [32]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
boolean_cols = df.select_dtypes(include=['bool']).columns.tolist()

print(f"\n1. Cột dạng số: {len(numeric_cols)}")
for i, col in enumerate(numeric_cols, 1):
    dtype = df[col].dtype
    print(f"   {i:2d}. {col:45s} ({dtype})")

print(f"\n2. Cột dạng phân loại: {len(categorical_cols)}")
for i, col in enumerate(categorical_cols, 1):
    n_unique = df[col].nunique()
    print(f"   {i:2d}. {col:45s} ({n_unique} giá trị duy nhất)")


1. Cột dạng số: 41
    1. age                                           (float64)
    2. appearances                                   (int64)
    3. minutes_played                                (int64)
    4. is_GK                                         (int64)
    5. is_DF                                         (int64)
    6. is_MF                                         (int64)
    7. is_FW                                         (int64)
    8. goals                                         (int64)
    9. assists                                       (int64)
   10. npg_per90                                     (float64)
   11. xag_per90                                     (float64)
   12. xg_per90                                      (float64)
   13. shots_per90                                   (float64)
   14. shots_on_target_per90                         (float64)
   15. shots_on_target_pct                           (float64)
   16. avg_shot_distance                           

#### **4. Kiểm tra kiểu dữ liệu không phù hợp**

In [33]:
inappropriate_issues = []

# 1. Chỉ số nhị phân
binary_cols = ['is_GK', 'is_DF', 'is_MF', 'is_FW']
for col in binary_cols:
    if col in df.columns:
        if df[col].dtype not in ['int64', 'int32']:
            inappropriate_issues.append(f"{col}: Nên là int, hiện tại là {df[col].dtype}")
            print(f"!  {col:40s}: Nên là int, hiện tại là {df[col].dtype}")
        
        unique_vals = df[col].dropna().unique()
        if not all(v in [0, 1] for v in unique_vals):
            inappropriate_issues.append(f"{col}: Chứa giá trị không phải nhị phân: {unique_vals}")
            print(f"x {col:40s}: Chứa giá trị không phải nhị phân!")

# 2. Cột phần trăm
percentage_cols = [col for col in df.columns if 'pct' in col.lower()]
for col in percentage_cols:
    if df[col].dtype in ['int64', 'float64']:
        min_val = df[col].min()
        max_val = df[col].max()
        if min_val < 0 or max_val > 100:
            inappropriate_issues.append(f"{col}: Ngoài khoảng 0-100% (min={min_val:.1f}, max={max_val:.1f})")
            print(f"!  {col:40s}: Ngoài khoảng 0-100% (min={min_val:.1f}, max={max_val:.1f})")

# 3. Số lưu dưới dạng text
for col in categorical_cols:
    if col not in ['nationality', 'position', 'position_category', 'current_club', 'league']:
        try:
            numeric_test = pd.to_numeric(df[col], errors='coerce')
            conversion_rate = numeric_test.notna().sum() / df[col].notna().sum() * 100
            
            if conversion_rate > 90:
                inappropriate_issues.append(f"{col}: {conversion_rate:.1f}% giá trị là số (lưu dạng text)")
                print(f"x {col:40s}: {conversion_rate:.1f}% giá trị là số (lưu dạng text)")
        except:
            pass

# 4. Cột hằng số
for col in categorical_cols:
    if df[col].nunique() == 1:
        inappropriate_issues.append(f"{col}: Chỉ có 1 giá trị duy nhất (cột hằng số)")
        print(f"!  {col:40s}: Chỉ có 1 giá trị duy nhất (cột hằng số)")

if not inappropriate_issues:
    print("\n- Tất cả kiểu dữ liệu đều phù hợp!")
else:
    print(f"\n- Tóm tắt: Tìm thấy {len(inappropriate_issues)} vấn đề về kiểu dữ liệu")


- Tất cả kiểu dữ liệu đều phù hợp!


#### **5. Phân bố giá trị**

##### **5.1. Các cột số**

In [34]:
if numeric_cols:
    numeric_stats = df[numeric_cols].describe().T
    numeric_stats['độ_lệch'] = [skew(df[col].dropna()) if df[col].notna().any() else np.nan 
                                  for col in numeric_cols]
    numeric_stats['độ_nhọn'] = [kurtosis(df[col].dropna()) if df[col].notna().any() else np.nan 
                                  for col in numeric_cols]
    numeric_stats['thiếu_%'] = [(df[col].isna().sum() / len(df) * 100) for col in numeric_cols]
    
    print("\n", numeric_stats.round(3).to_string())
    
    # Đặc điểm phân bố
    print("ĐẶC ĐIỂM PHÂN BỐ:")
    
    highly_skewed = []
    moderately_skewed = []
    symmetric = []
    
    for col in numeric_cols:
        sk = skew(df[col].dropna())
        if abs(sk) > 2:
            highly_skewed.append((col, sk))
        elif abs(sk) > 0.5:
            moderately_skewed.append((col, sk))
        else:
            symmetric.append((col, sk))
    
    if highly_skewed:
        print(f"\n🔴 Lệch cao (|độ lệch| > 2): {len(highly_skewed)} cột")
        for col, sk in sorted(highly_skewed, key=lambda x: abs(x[1]), reverse=True)[:10]:
            print(f"   • {col:45s}: {sk:7.3f}")
    
    if moderately_skewed:
        print(f"\n🟡 Lệch vừa phải (0.5 < |độ lệch| < 2): {len(moderately_skewed)} cột")
        for col, sk in sorted(moderately_skewed, key=lambda x: abs(x[1]), reverse=True)[:10]:
            print(f"   • {col:45s}: {sk:7.3f}")
    
    if symmetric:
        print(f"\n🟢 Gần đối xứng (|độ lệch| < 0.5): {len(symmetric)} cột")
        for col, sk in symmetric[:5]:
            print(f"   • {col:45s}: {sk:7.3f}")



                                   count      mean      std    min      25%      50%       75%      max  độ_lệch  độ_nhọn  thiếu_%
age                             16451.0    26.640    4.692  16.00   23.000   26.000    30.000    45.00    0.351   -0.363    0.012
appearances                     16453.0    17.210   11.164   0.00    7.000   17.000    27.000    49.00    0.012   -1.166    0.000
minutes_played                  16453.0  1126.022  916.573   0.00  285.000  978.000  1843.000  4223.00    0.502   -0.825    0.000
is_GK                           16453.0     0.090    0.286   0.00    0.000    0.000     0.000     1.00    2.872    6.246    0.000
is_DF                           16453.0     0.358    0.480   0.00    0.000    0.000     1.000     1.00    0.591   -1.651    0.000
is_MF                           16453.0     0.408    0.491   0.00    0.000    0.000     1.000     1.00    0.376   -1.859    0.000
is_FW                           16453.0     0.144    0.351   0.00    0.000    0.000     

##### **5.2. Các cột phân loại**

In [35]:
if categorical_cols:
    for col in categorical_cols:
        value_counts = df[col].value_counts()
        n_unique = df[col].nunique()
        
        print(f"\n{col.upper()}")
        print(f"   Số giá trị duy nhất: {n_unique}")
        print(f"   Thiếu: {df[col].isna().sum()} ({df[col].isna().sum()/len(df)*100:.1f}%)")
        print(f"   Top 5 giá trị:")
        for val, count in value_counts.head(5).items():
            pct = count / len(df) * 100
            print(f"      • {str(val):35s}: {count:5d} ({pct:5.1f}%)")


NATIONALITY
   Số giá trị duy nhất: 163
   Thiếu: 238 (1.4%)
   Top 5 giá trị:
      • Argentina                          :  1223 (  7.4%)
      • Brazil                             :  1095 (  6.7%)
      • Japan                              :   953 (  5.8%)
      • Spain                              :   707 (  4.3%)
      • Sweden                             :   560 (  3.4%)

POSITION
   Số giá trị duy nhất: 11
   Thiếu: 3 (0.0%)
   Top 5 giá trị:
      • MF                                 :  4493 ( 27.3%)
      • DF                                 :  4139 ( 25.2%)
      • FW                                 :  2371 ( 14.4%)
      • FW-MF                              :  2213 ( 13.5%)
      • DF-MF                              :  1515 (  9.2%)

POSITION_CATEGORY
   Số giá trị duy nhất: 5
   Thiếu: 0 (0.0%)
   Top 5 giá trị:
      • MF                                 :  6706 ( 40.8%)
      • DF                                 :  5897 ( 35.8%)
      • FW                                 :

#### **6. Giá trị thiếu**

In [36]:
missing_summary = pd.DataFrame({
    'Cột': df.columns,
    'Thiếu': df.isna().sum().values,
    'Thiếu_%': (df.isna().sum().values / len(df) * 100).round(2),
    'Không_thiếu': df.notna().sum().values
})

missing_summary = missing_summary.sort_values('Thiếu_%', ascending=False)

print(f"\nTóm tắt giá trị thiếu:")
print(f"   - Tổng số ô: {df.shape[0] * df.shape[1]:,}")
print(f"   - Số ô thiếu: {df.isna().sum().sum():,}")
print(f"   - Tỷ lệ thiếu: {(df.isna().sum().sum() / (df.shape[0] * df.shape[1]) * 100):.2f}%")

missing_cols = missing_summary[missing_summary['Thiếu'] > 0]

if len(missing_cols) > 0:
    print(f"\n-  {len(missing_cols)} cột có giá trị thiếu:")
    print("-" * 90)
    print(missing_cols.to_string(index=False))
else:
    print(f"\n- Không có giá trị thiếu trong dataset!")


Tóm tắt giá trị thiếu:
   - Tổng số ô: 756,838
   - Số ô thiếu: 330
   - Tỷ lệ thiếu: 0.04%

-  32 cột có giá trị thiếu:
------------------------------------------------------------------------------------------
                           Cột  Thiếu  Thiếu_%  Không_thiếu
                   nationality    238     1.45        16215
              key_passes_per90      3     0.02        16450
                      xg_per90      3     0.02        16450
  progressive_passes_rec_per90      3     0.02        16450
      progressive_passes_per90      3     0.02        16450
passes_into_penalty_area_per90      3     0.02        16450
 passes_into_final_third_per90      3     0.02        16450
           pass_completion_pct      3     0.02        16450
        passes_completed_per90      3     0.02        16450
                     gca_per90      3     0.02        16450
                     sca_per90      3     0.02        16450
             avg_shot_distance      3     0.02        16450
       

### **III. Kết luận**

### **1. Tổng quan dữ liệu**

**Điểm mạnh:**

- Kích thước phù hợp: **16 453** bản ghi với **46** thuộc tính
- Chất lượng cao: Chỉ **0.04%** dữ liệu thiếu **(330/756,838 ô)**, cho thấy tiền xử lý đã được thực hiện tốt
- Đa dạng thông tin: Bao gồm đầy đủ các nhóm chỉ số: thông tin cơ bản, tấn công, phòng thủ, chuyền bóng, rê dắt
- Phân bố vị trí hợp lý: Dữ liệu có đại diện đầy đủ cho 4 vị trí chính (GK, DF, MF, FW)

**Cần xem xét:**

- Phân bố không chuẩn: Biến mục tiêu (market_value) có độ lệch cao (skewness = 7.467)
- Nhiều features lệch: 7 cột có độ lệch cao, 24 cột có độ lệch vừa phải
- Outliers: Một số cầu thủ có giá trị thị trường rất cao (siêu sao) tạo nên outliers

### **2. Kiểu dữ liệu và giá trị**

- Tất cả cột có kiểu dữ liệu phù hợp.

- Các chỉ số nhị phân (is_GK, is_DF, is_MF, is_FW) đúng format 0/1.

- Giá trị phần trăm nằm trong 0-100%; không có giá trị bất thường.

### **3. Phân bố dữ liệu và outliers**

- Biến mục tiêu market_value lệch phải mạnh (skew = 7.467) do một số siêu sao.

- Một số features khác cũng lệch, cần xử lý trong modeling nếu dùng NN.

- Tree-based models xử lý tốt dữ liệu lệch và outliers.

#### **4. Gợi ý**

- Log transform target để giảm skewness.

- Feature engineering nâng cao (interaction, polynomial, threshold-based) thực hiện trong modeling.

- Lựa chọn thuật toán: Gradient Boosting / Random Forest ưu tiên; Neural Network cần preprocessing kỹ.

- Sử dụng K-Fold Cross-Validation để đảm bảo phân bố đều cho train/test.

#### **5. Kết luận tổng thể**

- Dataset sẵn sàng cho modeling: chất lượng dữ liệu cao, kiểu dữ liệu chính xác, đủ features và samples.

- Cần xử lý target biến lệch và lựa chọn thuật toán phù hợp.

- Insights từ dataset phản ánh thực tế bóng đá: đa số cầu thủ trung bình, một số ít siêu sao tạo outliers.